DIVIDE SUB FILE INTO CHUNKS TO INPUT TOKENS IN GPT FOR TRANSLATION

In [ ]:
import os

## RUN THIS CELL FIRST, THEN RUN THE TRANSLATION FILE
## YOUR SUB FILE NAME HERE!!
filename = "[UnkSubs] One Piece Film Z Eng Subs [720p].srt"

with open(filename, "r+", encoding="utf-8") as f:
    file_contents = f.read()

def fetch_chunk(data, start, end):
    try:
        return "\n".join(data[start:end])
    except Exception as e:
        print(f"Exception, returning data from start...\n{e}")
        return "\n".join(data[start:])

## DIVIDING FILE INTO CHUNKS OF MAXIMUM 200 LINES EACH
list_of_lines = file_contents.split('\n')
max_lines = 200
file_terminator = "\n"
## START OF EACH FILE
start = 0
## ITERATOR
item_count = 0
## COUNTER FOR HOW MANY LINES WE HAVE GONE TRHOUGH IN THAT FILE LOOP
line_count = 0
## POINTER TO LAST EMPTY LINE "\n"
last_break = 0
## COUNTER FOR FILE NUMBER
file_part = 0

output_dir= os.getcwd() + "\\fileparts"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

while item_count < len(list_of_lines):
    if list_of_lines[item_count] == "":
        last_break = item_count
    if (line_count > max_lines) or (item_count == len(list_of_lines)-1):
        with open(f"{output_dir}\\{filename.split('.')[0]}-{file_part}-engsub.txt", "w+", encoding="utf-8") as f:
            f.write(fetch_chunk(list_of_lines, start, last_break))
        start = last_break+1
        line_count = 0
        file_part = file_part + 1
        item_count = start
        continue
    line_count = line_count + 1
    item_count = item_count + 1

TRANSLATE THE CHUNKS USING GPT

In [ ]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
# Load the environment variables from .env file
load_dotenv()
# Accessing variables

def correct_text(text):
    """Sends the text to ChatGPT for correction."""
    
    client = OpenAI()

    response = client.chat.completions.create(
    model="gpt-4o",
    # response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": 
            """Você é um assistente muito útil de tradução de legendas. 
                Eu lhe informo um arquivo de legendas em inglês, e você traduz somente os textos da legenda, para português do Brasil.
                Você me retorna o texto traduzido na mesma formatação entregue, somente mudando o idioma da legenda."""},
        {"role": "user", "content": text}
    ]
    )

    return response.choices[0].message.content

## TRANSLATE SUB FILE FROM CHUNKS (STILL HAVE TO CHANGE ABSOLUTE PATH)
for i in range(file_part):
    ## IF YOU DONT HAVE INPUT FILES, RUN THE NOTEBOOK FOR CHUNKING BEFORE RUNNING THIS
    input_dir="fileparts"
    with open(f"{input_dir}\\{filename.split('.')[0]}-{i}-engsub.txt", "r", encoding="utf-8") as f:
        original_text = f.read()

    print(f"Starting translation part {i}...")
    translated_text = correct_text(original_text)

    output_dir= os.getcwd() + "\\translations"
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    with open(f"{output_dir}\\{filename.split('.')[0]}-pt-{i}-engsub.txt", "w+", encoding="utf-8") as f:
        f.write(translated_text)
    print(f"Translation part {i} finished!")


print("ALL Translation finished!")

UNIFY ALL CHUNK FILES INTO ONE SINGLE SOURCE FILE

In [44]:
import os

file_contents = ''
output_dir= os.getcwd() + "\\translations"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    
for i in range(file_part):
    with open(f"{output_dir}\\{filename.split('.')[0]}-pt-{i}-engsub.txt", "r", encoding="utf-8") as f:
        file_contents += '\n\n'+f.read()

with open(f"{output_dir}\\{filename.split('.')[0]}-FULL-PT-SUB.txt", "w+", encoding="utf-8") as f:
    f.write(file_contents)